In [32]:
import pandas as pd

df = pd.read_csv("data/boston.csv")
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48,22.0


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [34]:
import torch

X_train = df.iloc[:,:13].values
#X_train = torch.tensor(X_train)
X_train = torch.FloatTensor(X_train)

y_train = df.iloc[:,-1].values
y_train = torch.FloatTensor(y_train)


In [35]:
import torch.nn as nn
from torch.optim.adam import Adam

model = nn.Sequential(
    nn.Linear(13,100),
    nn.ReLU(),
    nn.Linear(100,50),
    nn.ReLU(),
    nn.Linear(50,1)
)

model

Sequential(
  (0): Linear(in_features=13, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=50, bias=True)
  (3): ReLU()
  (4): Linear(in_features=50, out_features=1, bias=True)
)

In [36]:
for name, param in model.named_parameters():
    print(f"Name: {name}")
    print(f"Shape: {param.shape}")
    print(f"Values: {param}\n")

Name: 0.weight
Shape: torch.Size([100, 13])
Values: Parameter containing:
tensor([[ 0.2411,  0.0898,  0.0007,  ...,  0.2103, -0.1179, -0.2117],
        [-0.2289,  0.1218, -0.0105,  ..., -0.1075,  0.2596, -0.1480],
        [-0.0396,  0.1829,  0.1489,  ...,  0.2283,  0.1598, -0.0970],
        ...,
        [-0.1950,  0.1136,  0.2386,  ..., -0.2618, -0.0892,  0.1496],
        [-0.2264,  0.2548, -0.2134,  ...,  0.0022, -0.1483, -0.0138],
        [ 0.0780, -0.0208, -0.0980,  ..., -0.0766, -0.1924, -0.2324]],
       requires_grad=True)

Name: 0.bias
Shape: torch.Size([100])
Values: Parameter containing:
tensor([-1.5356e-01, -9.2209e-03, -1.4203e-01,  6.4857e-02, -7.4510e-02,
         1.9467e-01,  4.2868e-02, -2.1670e-01,  3.7495e-02,  1.7886e-01,
        -5.2199e-02,  2.3551e-01,  4.1251e-02,  1.9002e-01, -2.1150e-01,
        -2.2807e-01,  8.4788e-05, -3.7539e-02,  2.7594e-01,  1.4317e-03,
        -1.3753e-01,  3.8800e-02, -9.9853e-02, -6.0627e-02, -8.0796e-02,
         6.6171e-02, -2.5179e-0

In [37]:
optim = Adam(model.parameters(), lr=0.001)
epochs = 200
criterion = nn.MSELoss()

for epoch in range(epochs):
    # 순서가 정말 중요!!!
    optim.zero_grad() 
    y_pred = model(X_train)
    loss = criterion(y_pred,y_train)
    loss.backward()
    optim.step()

    print("loss : ",  loss.item())


loss :  1586.50341796875
loss :  1029.7838134765625
loss :  620.5833740234375
loss :  339.564697265625
loss :  168.02198791503906
loss :  97.31220245361328
loss :  104.77114868164062
loss :  157.63475036621094
loss :  218.53753662109375
loss :  259.8060607910156
loss :  272.995849609375
loss :  263.9592590332031
loss :  238.94732666015625
loss :  205.49159240722656
loss :  170.75180053710938
loss :  141.22459411621094
loss :  119.09840393066406
loss :  104.12287139892578
loss :  95.29659271240234
loss :  91.86137390136719
loss :  92.5843276977539
loss :  96.10360717773438
loss :  101.03744506835938
loss :  106.12104034423828
loss :  110.35691833496094
loss :  113.06779479980469
loss :  113.92658233642578
loss :  112.94369506835938
loss :  110.41405487060547
loss :  106.81401062011719
loss :  102.70881652832031
loss :  98.64321899414062
loss :  95.08275604248047
loss :  92.35115814208984
loss :  90.60894012451172
loss :  89.8493881225586
loss :  89.92745971679688
loss :  90.598869323730

c:\POTENUP\08_Deep_Learning\.venv\Lib\site-packages\torch\nn\modules\loss.py:616: UserWarning: Using a target size (torch.Size([506])) that is different to the input size (torch.Size([506, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss :  88.88709259033203
loss :  88.78242492675781
loss :  88.58491516113281
loss :  88.32990264892578
loss :  88.05941772460938
loss :  87.81281280517578
loss :  87.6208267211914
loss :  87.4986343383789
loss :  87.44623565673828
loss :  87.45081329345703
loss :  87.49097442626953
loss :  87.54231262207031
loss :  87.58311462402344
loss :  87.59829711914062
loss :  87.58135986328125
loss :  87.53426361083984
loss :  87.4656982421875
loss :  87.38807678222656
loss :  87.31415557861328
loss :  87.25418090820312
loss :  87.21401977539062
loss :  87.1944808959961
loss :  87.19204711914062
loss :  87.20023345947266
loss :  87.21165466308594
loss :  87.21971893310547
loss :  87.21991729736328
loss :  87.21054077148438
loss :  87.19265747070312
loss :  87.16927337646484
loss :  87.14434051513672
loss :  87.12162017822266
loss :  87.10387420654297
loss :  87.09223175048828
loss :  87.08634948730469
loss :  87.0845947265625
loss :  87.08473205566406
loss :  87.08445739746094
loss :  87.082069

In [38]:
model.state_dict()

OrderedDict([('0.weight',
              tensor([[ 0.2360,  0.0744,  0.0039,  ...,  0.2136, -0.1227, -0.2080],
                      [-0.2157,  0.1123, -0.0159,  ..., -0.1159,  0.2517, -0.1533],
                      [-0.0377,  0.1727,  0.1406,  ...,  0.2189,  0.1520, -0.1053],
                      ...,
                      [-0.1867,  0.1056,  0.2316,  ..., -0.2703, -0.0946,  0.1431],
                      [-0.2264,  0.2548, -0.2134,  ...,  0.0022, -0.1483, -0.0138],
                      [ 0.0674, -0.0208, -0.1106,  ..., -0.0894, -0.2035, -0.2449]])),
             ('0.bias',
              tensor([-0.1491, -0.0186, -0.1524,  0.0742, -0.0700,  0.1973,  0.0520, -0.2286,
                       0.0252,  0.1695, -0.0522,  0.2355,  0.0312,  0.1900, -0.2246, -0.2404,
                      -0.0182, -0.0375,  0.2834, -0.0093, -0.1375,  0.0388, -0.0739, -0.0540,
                      -0.0808,  0.0726, -0.2612,  0.2615, -0.2256, -0.1523,  0.1079,  0.1120,
                      -0.1501, -0.1398, 

In [39]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [40]:
X_train = X_train.to(device)
y_train = y_train.to(device)
model = model.to(device)

optim = Adam(model.parameters(), lr=0.001)
epochs = 200
criterion = nn.MSELoss()

for epoch in range(epochs):
    # 순서가 정말 중요!!!
    optim.zero_grad() 
    y_pred = model(X_train)
    loss = criterion(y_pred,y_train)
    loss.backward()
    optim.step()

    print("loss : ",  loss.item())


loss :  86.3820571899414
loss :  86.65125274658203
loss :  86.52328491210938
loss :  89.90563201904297
loss :  87.09339141845703
loss :  89.5189208984375
loss :  86.77336120605469
loss :  86.81427764892578
loss :  88.33548736572266
loss :  87.11885833740234
loss :  86.29568481445312
loss :  87.24727630615234
loss :  87.32537841796875
loss :  86.34407806396484
loss :  86.27946472167969
loss :  86.95538330078125
loss :  86.80448913574219
loss :  86.21959686279297
loss :  86.3359375
loss :  86.70252227783203
loss :  86.45482635498047
loss :  86.08718872070312
loss :  86.24908447265625
loss :  86.4614486694336
loss :  86.2509536743164
loss :  86.07780456542969
loss :  86.2396469116211
loss :  86.29861450195312
loss :  86.0916748046875
loss :  86.02745056152344
loss :  86.16889953613281
loss :  86.15418243408203
loss :  86.01823425292969
loss :  86.04227447509766
loss :  86.11329650878906
loss :  86.03031158447266
loss :  85.95833587646484
loss :  86.01771545410156
loss :  86.02974700927734